# Closures
A closure is a function object that is able to remember its environment or use values in enclosing scoping that are not in the memory anymore.

## Nested functions
A function defined inside another function is known as a ***nested function***.
The names defined in the outer function scope are ***nonlocal*** from the nested/inner function point of view.
The nested funcion is able to access to the ***nonlocal*** variables, but it is **not** able to modify them without the nonlocal statement.

In [1]:
def outer_function(text: str):
    def inner_function():
        # text is in the outer_function scope, then it is nonlocal from the inner_function point of view
        # inner_function is a nested funciton of outer_function, then it is able to accesses to text variable
        print(text)
    return inner_function()
outer_function("Hello World")

Hello World


In [2]:
def outer_function(text: str):
    def inner_function():
        # inner_function is able to accesses to text variable but it is not able to modify it
        text += " from inner function"
        print(text)
    return inner_function()
try:
    outer_function("Hello World")
except UnboundLocalError as exc:
    print("ERROR - UnboundLocalError:", exc)

ERROR - UnboundLocalError: local variable 'text' referenced before assignment


In [3]:
def outer_function(text: str):
    def inner_function():
        # to modify variables in the outer_function scope it is required the nonlocal statement
        nonlocal text
        text += " from inner function"
        print(text)
    return inner_function()
outer_function("Hello World")

Hello World from inner function


In [4]:
def outer_function(text: str):
    text = " This is outer function"
    def inner_function():
        text = " This is inner function"
        def second_inner_function():
            # it looks nonlocal statement takes the closest upper scope
            nonlocal text
            text += " and this is second inner function"
            print(text)
        return second_inner_function()
    return inner_function()
outer_function("Hello World")

 This is inner function and this is second inner function


## Closure usage
To obtain a closure we need to return the nested function without calling it, because we want a function not the function result. Then, we have the following use cases
- Replace hardcoded constants, they can be part of the outer scope and be used in the inner scope
- Reduce use of global variables
- Data hiding, it works as callback function
It is useful to work with closures to add functionalities to a function and we don't need/want to define classes

### Example

In [5]:
# Let's start with the following example
translations = {
        ("es", "I'm a python developer"): "Yo soy un programador python",
        ("en", "Yo soy un programador python"): "I'm a python developer",
    }

def translate(text, lang):
    try:
        return translations[(lang, text)]
    except KeyError:
        return text
print(translate("I'm a python developer", "es"))
print(translate("Yo soy un programador python", "en"))

Yo soy un programador python
I'm a python developer


In [6]:
# Now, we want to:
# - Reduce use of global variables
# - Data hiding
def translate_setup():
    # translations is not global variable anymore and it is hide as well
    translations = {
        ("es", "I'm a python developer"): "Yo soy un programador python",
        ("en", "Yo soy un programador python"): "I'm a python developer",
    }
    def translate(text, lang):
        try:
            return translations[(lang, text)]
        except KeyError:
            return text
    return translate

# translate_func stores the translate function with 'translations' variable from translate_setup scope that already completes
translate_func = translate_setup()
print(translate_func("I'm a python developer", "es"))
print(translate_func("Yo soy un programador python", "en"))

Yo soy un programador python
I'm a python developer


In [7]:
# Now we want to:
# - Replace hardcoded constants
def translate_setup(lang):
    translations = {
        ("es", "I'm a python developer"): "Yo soy un programador python",
        ("en", "Yo soy un programador python"): "I'm a python developer",
    }
    # lang is from the outer_scope and current_lang is from the inner_scope. We can read lang if the closure is called without current_lang
    def translate(text, current_lang=lang): 
        try:
            return translations[(current_lang, text)]
        except KeyError:
            return text
    return translate

# We assign 'es' to lang that is part of the outer/translate_setup scope
translate_func = translate_setup("es")
# The closure (translate_func) will use the lang in the outer_scope because we are not providing a current_lang
print(translate_func("I'm a python developer"))
# We can define a current_lang as well. In this scenario, the closure uses 'en' instead of lang
print(translate_func("Yo soy un programador python", "en"))

Yo soy un programador python
I'm a python developer


## How does it really work?
The outer_function variables are shared between the outer_scope and inner_scope, this is possible because python creates an intermediate object called a ***cell***. Then, a closure could be considered as a function with an additional variable that contains some variables.

In [8]:
# Let's assume that we have two variables in the outer_scope: 'translations' and 'lang'
def translate_setup():
    lang = "es"
    translations = {
        ("es", "I'm a python developer"): "Yo soy un programador python",
        ("en", "Yo soy un programador python"): "I'm a python developer",
    }
    print("Outer Scope -", hex(id(translations)))
    def translate(text, current_lang): 
        try:
            print("Inner Scope -", hex(id(translations)))
            return translations[(current_lang, text)]
        except KeyError:
            return text
    return translate

translate_func = translate_setup()
translate_func("I'm a python developer", "es")

# __closure__ is the cell object that stores the 'translations' memory location
# We can see that the translations object is the same in the outer scope, inner scope and cell object
print(translate_func.__closure__)
print(translate_func.__code__.co_freevars)

# We only see the translations object in the cell object because is the only variable used in the outer/inner scope
# The lang variable is not used

Outer Scope - 0x7f72c8670300
Inner Scope - 0x7f72c8670300
(<cell at 0x7f72c8677bb0: dict object at 0x7f72c8670300>,)
('translations',)


In [9]:
# Let's use lang in the outer and inner scope
def translate_setup():
    lang = "es"
    translations = {
        ("es", "I'm a python developer"): "Yo soy un programador python",
        ("en", "Yo soy un programador python"): "I'm a python developer",
    }
    def translate(text, current_lang=None): 
        try:
            if current_lang is None:
                current_lang = lang
            return translations[(current_lang, text)]
        except KeyError:
            return text
    return translate

translate_func = translate_setup()

# __closure__ shows the lang and translation memory positions, because they are used in the outer/inner scope
print(translate_func.__closure__)
print(translate_func.__code__.co_freevars)

(<cell at 0x7f72c8677400: str object at 0x7f72ca0c6c70>, <cell at 0x7f72c8677730: dict object at 0x7f72c8623600>)
('lang', 'translations')
